# ***Analysis of Germany COVID-19 pandemic***
## My hands on try for using python for data analytics and visualizations. Mostly compare the data in relation to the gender, but although other analytics where made.
## Titles could be formatted in a more uniform way...it come's to my to-do list for the next analysis...I promise! ;) 

# Import Dataset from Kaggle
### Kudos to headsortails!!

In [ ]:
import pandas as pd
data=pd.read_csv('../input/covid19-tracking-germany/covid_de.csv')
df = pd.DataFrame(data)
df['date']=pd.to_datetime(df['date'].astype(str), format='%Y-%m-%d') 
sorted_df = df.sort_values(by=["date"], ascending=True)

# Import packages and functions

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import scipy
import string
import math
import seaborn as sns
from scipy.stats import norm
from statsmodels.stats.diagnostic import lilliefors
from statsmodels.graphics.gofplots import qqplot
from scipy.stats import norm, kurtosis, kurtosistest, pearsonr, skew, normaltest, chisquare, chi2_contingency, chi2
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import datetime as dt #Not used!

from matplotlib import pyplot
#import cupy as cp #Not available 

#Get DOF with least RMSE
## Kudos to jonnybazookatone: https://stackoverflow.com/questions/47442102/how-to-find-the-best-degree-of-polynomials

def get_degree (a,b):
    X = a
    y = b

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    rmses = []
    degrees = np.arange(1, 100)
    min_rmse, min_deg = 1e10, 0

    for deg in degrees:

    # Train features
        poly_features = PolynomialFeatures(degree=deg, include_bias=False)
        x_poly_train = poly_features.fit_transform(x_train)

    # Fit data
        poly_reg = LinearRegression()
        poly_reg.fit(x_poly_train, y_train)

    # Get RMSE of test data
        x_poly_test = poly_features.fit_transform(x_test)
        poly_predict = poly_reg.predict(x_poly_test)
        poly_mse = mean_squared_error(y_test, poly_predict)
        poly_rmse = np.sqrt(poly_mse)
        rmses.append(poly_rmse)
    
    # CV
        if min_rmse > poly_rmse:
            min_rmse = poly_rmse
            min_deg = deg

# Print calculated DOF
    print('Best DOF {} with RMSE {}'.format(min_deg, min_rmse))
        
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.plot(degrees, rmses)
    ax.set_yscale('log')
    ax.set_xlabel('Degree')
    ax.set_ylabel('RMSE')
    return min_deg

# Normalize and clean dataset

In [ ]:
df['date']=pd.to_datetime(df['date'].astype(str), format='%Y-%m-%d') 
sorted_df = df.sort_values(by=["date"], ascending=True)
sorted_df = sorted_df.dropna()
sorted_df = sorted_df.reset_index(drop=True)

print('Entities pre cleanup %d'%df.shape[0])
print('Entities post cleanup %d'%sorted_df.shape[0])
print('Removed entities %d'%(df.shape[0]-sorted_df.shape[0]))
print('Percentage removed %f'%((100/df.shape[0])*(df.shape[0]-sorted_df.shape[0])))

# Quick representation of dataset

In [ ]:
sorted_df.head(5)

In [ ]:
sorted_df.describe(include = [np.number])

In [ ]:
sorted_df.describe(include=['object'])

In [ ]:
print('Cases total: {} '.format(sorted_df['cases'].sum())) #Summe
print('Deaths total: {} '.format(sorted_df['deaths'].sum())) #Summe
print('Recoveries total: {} '.format(sorted_df['recovered'].sum())) #Summe

In [ ]:
cinf=sorted_df['deaths'].sum()+sorted_df['recovered'].sum()
aktiv=(sorted_df['cases'].sum()-cinf)
print('Currently infected: {}'.format(aktiv))

# Split dataset for gender specific sub-datasets

In [ ]:
df_male = sorted_df['gender'] == "M"
df_female = sorted_df['gender'] == "F"
df_male = sorted_df[df_male]
df_female = sorted_df[df_female]
df_male=df_male.reset_index(drop=True)
df_female=df_female.reset_index(drop=True)


# Aggregate daily reports for unique day indexes for main dataset and sub datasets

In [ ]:
unique_f = df_female['date'].unique() 
unique = sorted_df['date'].unique()
unique_m = df_male['date'].unique()

# Female Dataset
dates_cases_f = []
for x in unique_f:
    filter = np.where(df_female['date']==x)[0]
    
    dates_cases_f.append((np.sum(df_female['cases'][filter]))) 

dates_deaths_f = []
for x in unique_f:
    filter = np.where(df_female['date']==x)[0]
    
    dates_deaths_f.append((np.sum(df_female['deaths'][filter]))) 

dates_recovered_f = []
for x in unique_f:
    filter = np.where(df_female['date']==x)[0]
    
    dates_recovered_f.append((np.sum(df_female['recovered'][filter]))) 

# Male dataset
dates_cases_m = []
for x in unique_m:
    filter = np.where(df_male['date']==x)[0]
    
    dates_cases_m.append((np.sum(df_male['cases'][filter]))) 


dates_deaths_m = []
for x in unique_m:
    filter = np.where(df_male['date']==x)[0]
    
    dates_deaths_m.append((np.sum(df_male['deaths'][filter]))) 


dates_recovered_m = []

for x in unique_m: 
    filter = np.where(df_male['date']==x)[0]
    
    dates_recovered_m.append((np.sum(df_male['recovered'][filter]))) 

# Main dataset
dates_cases = []

for x in unique:
    filter = np.where(sorted_df['date']==x)[0]
    
    dates_cases.append((np.sum(sorted_df['cases'][filter]))) 

dates_death = []
for x in unique: 
    filter = np.where(sorted_df['date']==x)[0] 

    dates_death.append((np.sum(sorted_df['deaths'][filter]))) 

dates_recovered = []

for x in unique: #
    filter = np.where(sorted_df['date']==x)[0]
    
    dates_recovered.append((np.sum(sorted_df['recovered'][filter]))) 


In [ ]:
print('Analytics involving {} days'.format(np.array(dates_cases).size))
print('First day (Y-D-M): {}'.format(unique[0]))
print('Last day (Y-D-M): {}'.format(unique[unique.size-1]))
print('')
print('On average {} cases per day in Germany'.format(np.mean(dates_cases)))
print('On average {} deaths per day in Germany'.format(np.mean(dates_death)))
print('On average {} recoveries per day in Germany'.format(np.mean(dates_recovered)))

In [ ]:
print('Mortality overall: %f'%((100/sorted_df['cases'].sum())*sorted_df['deaths'].sum()))
print('Mortality females: %f'%((100/df_female['cases'].sum())*df_female['deaths'].sum()))
print('Mortality males: %f'%((100/df_male['cases'].sum())*df_male['deaths'].sum()))

### => Males are dying sligthly more 

# **1. Graphical analysis of data distributions**

# Plot distribution of reported cases for main dataset

In [ ]:
%matplotlib inline
plt.figure(figsize=(20,20))
sns.distplot(sorted_df['cases'], kde=False)

# Plot distribution of reported deaths for main dataset

In [ ]:
%matplotlib inline
plt.figure(figsize=(20,20))
sns.distplot(sorted_df['deaths'], kde=False)

# Plot distribution of reported recoveries for main dataset

In [ ]:
%matplotlib inline
plt.figure(figsize=(20,20))
sns.distplot(sorted_df['recovered'], kde=False)

## Plot mean of reported cases for each state. Compare against grand-mean

In [ ]:
unique = sorted_df['state'].unique()
classes_mean = []
classes = []
for states in unique: 
    filter = np.where(sorted_df['state']==states)[0]
    print (states)
    classes_mean.append(np.average((sorted_df['cases'][filter]))) 
classes_mean=np.array(classes_mean)
classes=np.array(classes)

In [ ]:
plt.figure(figsize = (20,20))
height = np.sort(classes_mean)
bars = unique[np.argsort(classes_mean)]
y_pos = np.arange(len(bars))
avg = np.mean(classes_mean)
plt.axhline(avg, color='k', linestyle='dashed', linewidth=1)
plt.text(10, avg, avg, fontsize=20, va='center', ha='center', backgroundcolor='w')

plt.bar(y_pos, height)
 

plt.title('Mean of daily reported cases per state')
plt.xlabel('State')
plt.ylabel('Mean of cases per day')
 
plt.ylim(0,20)
 
plt.xticks(y_pos, bars, rotation=90)

plt.show()

## Gender ratio of reported cases 

In [ ]:
sum_female=sum(np.array(df_female['cases']))
sum_male=sum(np.array(df_male['cases']))

#Array bauen
gender_counts = sorted_df['gender'].value_counts()
gender_counts[0]=sum_female
gender_counts[1] = sum_male
df2 = pd.DataFrame({'Cases ratio': gender_counts}, 
                     index = ['M', 'F']
                   )
df2.plot.pie(y='Cases ratio', figsize=(10,10), autopct='%1.1f%%')

## Gender ratio of reported deaths

In [ ]:
sum_female_death=sum(np.array(df_female['deaths']))
sum_male_death=sum(np.array(df_male['deaths']))

gender_counts = sorted_df['gender'].value_counts()
gender_counts[0]=sum_female_death
gender_counts[1] = sum_male_death
df2 = pd.DataFrame({'Deaths ratio': gender_counts}, 
                     index = ['M', 'F']
                   )
df2.plot.pie(y='Deaths ratio', figsize=(10,10), autopct='%1.1f%%')

## Gender ratio of reported recoveries

In [ ]:
sum_female_rec=sum(np.array(df_female['recovered']))
sum_male_rec=sum(np.array(df_male['recovered']))

gender_counts = sorted_df['gender'].value_counts()
gender_counts[0]=sum_female_rec
gender_counts[1] = sum_male_rec
df2 = pd.DataFrame({'Recovery ratio': gender_counts}, 
                     index = ['M', 'F']
                   )
df2.plot.pie(y='Recovery ratio', figsize=(10,10), autopct='%1.1f%%')

# Time domain analysis with different hues
## Cases in relation to age group (CI = STD)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
sns.set_style("darkgrid")
sns.lineplot(ax=ax,data=sorted_df, x="date", y="cases", hue="age_group", ci='sd', hue_order=['80-99','60-79','35-59','15-34','05-14','00-04'])

## Cases in relation to age group (CI = 0.95)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
sns.set_style("darkgrid")
sns.lineplot(ax=ax,data=sorted_df, x="date", y="cases", hue="age_group", ci=95, hue_order=['80-99','60-79','35-59','15-34','05-14','00-04'])

## Cases in relation to gender (CI = STD)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
sns.set_style("darkgrid")
sns.lineplot(ax=ax,data=sorted_df, x="date", y="cases", hue="gender", ci='sd')

## Cases in relation to gender (CI = 0.95)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
sns.set_style("darkgrid")
sns.lineplot(ax=ax,data=sorted_df, x="date", y="cases", hue="gender", ci=95)


## Deaths in relation to age group (CI = STD)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
sns.set_style("darkgrid")
sns.lineplot(ax=ax,data=sorted_df, x="date", y="deaths", hue="age_group", ci='sd', hue_order=['80-99','60-79','35-59','15-34','05-14','00-04'])

## Deaths in relation to age group (CI = 0.95)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
sns.set_style("darkgrid")
sns.lineplot(ax=ax,data=sorted_df, x="date", y="deaths", hue="age_group", ci=95, hue_order=['80-99','60-79','35-59','15-34','05-14','00-04'])

## Deaths in relation to gender (CI = STD)


In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
sns.set_style("darkgrid")
sns.lineplot(ax=ax,data=sorted_df, x="date", y="deaths", hue="gender", ci='sd')

## Deaths in relation to gender (CI = 0.95)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
sns.set_style("darkgrid")
sns.lineplot(ax=ax,data=sorted_df, x="date", y="deaths", hue="gender", ci=95)

## Recoveries in relation to age group (CI = STD)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
sns.set_style("darkgrid")
sns.lineplot(ax=ax,data=sorted_df, x="date", y="recovered", hue="age_group", ci='sd', hue_order=['80-99','60-79','35-59','15-34','05-14','00-04'])

## Recoveries in relation to age group (CI = 0.95)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
sns.set_style("darkgrid")
sns.lineplot(ax=ax,data=sorted_df, x="date", y="recovered", hue="age_group", ci=95, hue_order=['80-99','60-79','35-59','15-34','05-14','00-04'])

## Recoveries in relation to gender (CI = STD)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
sns.set_style("darkgrid")
sns.lineplot(ax=ax,data=sorted_df, x="date", y="recovered", hue="gender", ci='sd')

## Recoveries in relation to age group (CI = 0.95)

In [ ]:
fig, ax = plt.subplots(figsize = (20, 15))
sns.set_style("darkgrid")
sns.lineplot(ax=ax,data=sorted_df, x="date", y="recovered", hue="gender", ci=95)

# **Check data distributions of non aggregated main dataset**
## Plot QQ-Plot

## Cases

In [ ]:
qqplot(sorted_df['cases'], line='s') 
pyplot.show()

## Deaths

In [ ]:
qqplot(sorted_df['deaths'], line='s') 
pyplot.show()

## Recoveries

In [ ]:
qqplot(sorted_df['recovered'], line='s') 
pyplot.show()

# **Check data distributions of aggregated main dataset**
## Cases

In [ ]:
qqplot(np.array(dates_cases), line='s') 
pyplot.show()

## Deaths

In [ ]:
qqplot(np.array(dates_death), line='s') 
pyplot.show()

## Recoveries

In [ ]:
qqplot(np.array(dates_recovered), line='s') 
pyplot.show()

### => Data distirbutions are bimodal distributed

# Plot distribution of aggregated reported cases for main dataset
## Cases

In [ ]:
%matplotlib inline
plt.figure(figsize=(20,20))
sns.distplot(dates_cases, kde=False)

## Deaths

In [ ]:
%matplotlib inline
plt.figure(figsize=(20,20))
sns.distplot(dates_death, kde=False)

## Recoveries

In [ ]:
%matplotlib inline
plt.figure(figsize=(20,20))
sns.distplot(dates_recovered, kde=False)

# **Plot infection waves from aggregated reported cases data**
## Format dates to index

In [ ]:
unique = sorted_df['date'].unique() #unique value was gone?
poly_x = unique.reshape(-1,1) #poly_x is used later
day_index = np.arange(0,poly_x.size,1)
day_index = day_index.reshape(-1,1)
day_index.shape

days_f = unique_f.reshape(-1,1)
day_index_f = np.arange(0,days_f.size,1)
day_index_f = day_index_f.reshape(-1,1)
day_index_f.shape

days_m = unique_m.reshape(-1,1)
day_index_m = np.arange(0,days_m.size,1)
day_index_m = day_index_m.reshape(-1,1)
day_index_m.shape

In [ ]:
%matplotlib inline
plt.rcParams.update(plt.rcParamsDefault)
plt.clf()
fig, ax = plt.subplots(figsize = (30, 15))
sns.lineplot(ax=ax, x=np.arange(0,day_index.size,1), y=dates_cases)

plt.vlines(x=30, ymin=dates_cases[30], ymax=40000, label="Prob. wave 1",color='k', linestyle='dashed', linewidth=2)
plt.text(10, 40000, 'Prob. wave 1', fontsize=20, va='center', ha='center', backgroundcolor='w')


plt.vlines(x=230, ymin=dates_cases[230], ymax=40000, label="Prob. wave 2",color='k', linestyle='dashed', linewidth=2)
plt.text(210, 40000, 'Prob. wave 2', fontsize=20, va='center', ha='center', backgroundcolor='w')

plt.vlines(x=380, ymin=dates_cases[380], ymax=40000, label="Prob. wave 3",color='k', linestyle='dashed', linewidth=2)
plt.text(360, 40000, 'Prob. wave 3', fontsize=20, va='center', ha='center', backgroundcolor='w')


# !!!Start of waves based on own interpretations! More precise identification at a later time!!!

In [ ]:
print('Start wave 1 ca. {}'.format(unique[30]))
print('Start wave 2 ca. {}'.format(unique[230]))
print('Start wave 3 ca. {}'.format(unique[380]))

# **Advanced analysis of wave distribution by using polynomial regression**
## Get best degree of freedom by simulating differend DOF-values mentioned at the beginning

## Regression cases

In [ ]:
poly_reg = PolynomialFeatures(degree=get_degree(day_index,dates_cases))
X_poly = poly_reg.fit_transform(day_index)
pol_reg = LinearRegression()
pol_reg.fit(X_poly, dates_cases)

fig, ax = plt.subplots(figsize = (20, 15))

def viz_polymonial():
    plt.scatter(day_index, dates_cases, color='red')
    plt.plot(day_index, pol_reg.predict(poly_reg.fit_transform(day_index)), color='blue')
    plt.title('Polynomial Reg')
    plt.xlabel('Time index')
    plt.ylabel('Reported cases')
    plt.show()
    return
viz_polymonial()

## Regression Deaths

In [ ]:
poly_reg = PolynomialFeatures(degree=get_degree(day_index,dates_death))
X_poly = poly_reg.fit_transform(day_index)
pol_reg = LinearRegression()
pol_reg.fit(X_poly, dates_death)

fig, ax = plt.subplots(figsize = (20, 15))

def viz_polymonial():
    plt.scatter(day_index, dates_death, color='red')
    plt.plot(day_index, pol_reg.predict(poly_reg.fit_transform(day_index)), color='blue')
    plt.title('Polynomial Reg')
    plt.xlabel('Time index')
    plt.ylabel('Reported deaths')
    plt.show()
    return
viz_polymonial()

## Regression Recoveries

In [ ]:
poly_reg = PolynomialFeatures(degree=get_degree(day_index,dates_recovered))
X_poly = poly_reg.fit_transform(day_index)
pol_reg = LinearRegression()
pol_reg.fit(X_poly, dates_recovered)

fig, ax = plt.subplots(figsize = (20, 15))

def viz_polymonial():
    plt.scatter(day_index, dates_recovered, color='red')
    plt.plot(day_index, pol_reg.predict(poly_reg.fit_transform(day_index)), color='blue')
    plt.title('Polynomial Reg')
    plt.xlabel('Time index')
    plt.ylabel('Reported recoveries')
    plt.show()
    return
viz_polymonial()

# !!!Predicted fourth wave mainly due to high DOF! More data will show...!!!

## High variance of datapoints after 2nd wave results in skip of 1st wave
## Regression imply that 2nd wave started way before day 230

## Calculate local minima between 2nd and 3rd wave to predict start of 3th wave more precisely

In [ ]:
print('Start wave 3 {}'.format(unique[np.where(pol_reg.predict(poly_reg.fit_transform(day_index)) == np.min(pol_reg.predict(poly_reg.fit_transform(day_index))[350:400]))][0]))

## Calculate local minima between 3rd and 4th wave to predict "start" of 4th wave more precisely

In [ ]:
print('Start wave 4 {}'.format(unique[np.where(pol_reg.predict(poly_reg.fit_transform(day_index)) == np.min(pol_reg.predict(poly_reg.fit_transform(day_index))[450:481]))][0]))

# **Use regression with tight time domain index from day 0 to day 150 for more precise analytic of 1st wave**

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly_reg = PolynomialFeatures(degree=get_degree(day_index[:150],dates_recovered[:150]))
X_poly = poly_reg.fit_transform(day_index[:150])
pol_reg = LinearRegression()
pol_reg.fit(X_poly, dates_cases[:150])

fig, ax = plt.subplots(figsize = (20, 15))

# Visualizing the Polymonial Regression results
def viz_polymonial():
    plt.scatter(day_index[:150], dates_cases[:150], color='red')
    plt.plot(day_index[:150], pol_reg.predict(poly_reg.fit_transform(day_index[:150])), color='blue')
    plt.axhline(np.mean(np.array(dates_cases[:150])), color='k', linestyle='dashed', linewidth=1) #Integration des horizontalen Mittelwertes
    plt.text(110, np.mean(np.array(dates_cases[:150])), np.mean(np.array(dates_cases[:150])), fontsize=20, va='center', ha='center', backgroundcolor='w') #Textdefinition Mittelwert
    plt.title('Polynomial regression cases 1st wave')
    plt.xlabel('Time index')
    plt.ylabel('Reported cases')
    plt.show()
    return
viz_polymonial()

## There is a noticeable exceed of the mean from between day 20-30 until day 80. Probably our first wave.

# **Advanced analysis of gender specific wave distributions by using polynomial regression**

## Compare regression cases M/F

In [ ]:
get_degree(day_index_m,dates_cases_m) #Sub-Dataset male

In [ ]:
get_degree(day_index_f,dates_cases_f) #Sub-Dataset female

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly_reg = PolynomialFeatures(degree=14) #Use same DOF cause optimal DOF is identical

X_poly_f = poly_reg.fit_transform(day_index_f)
pol_reg_f = LinearRegression()
pol_reg_f.fit(X_poly_f, dates_cases_f)

X_poly_m = poly_reg.fit_transform(day_index_m)
pol_reg_m = LinearRegression()
pol_reg_m.fit(X_poly_m, dates_cases_m)

fig, ax = plt.subplots(figsize = (20, 15))

def viz_polymonial():
    plt.scatter(day_index_f, dates_cases_f, color='red')
    plt.plot(day_index_f, pol_reg_f.predict(poly_reg.fit_transform(day_index_f)), color='red')
    plt.scatter(day_index_m, dates_cases_m, color='blue')
    plt.plot(day_index_m, pol_reg_m.predict(poly_reg.fit_transform(day_index_m)), color='blue')
    plt.title('Polynomial regression cases M/F')
    plt.xlabel('Time Index')
    plt.ylabel('Reported Cases')
    plt.show()
    return
viz_polymonial()

## Compare regression deaths M/F
### Different DOF's indicated in earlier calculations

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly_reg_m = PolynomialFeatures(degree=get_degree(day_index_m,dates_deaths_m))
poly_reg_f = PolynomialFeatures(degree=get_degree(day_index_f,dates_deaths_f))

X_poly_f = poly_reg_f.fit_transform(day_index_f)
pol_reg_f = LinearRegression()
pol_reg_f.fit(X_poly_f, dates_deaths_f)

X_poly_m = poly_reg_m.fit_transform(day_index_m)
pol_reg_m = LinearRegression()
pol_reg_m.fit(X_poly_m, dates_deaths_m)

fig, ax = plt.subplots(figsize = (20, 15))

# Visualizing the Polymonial Regression results
def viz_polymonial():
    plt.scatter(day_index_f, dates_deaths_f, color='red')
    plt.plot(day_index_f, pol_reg_f.predict(poly_reg_f.fit_transform(day_index_f)), color='red')
    plt.scatter(day_index_m, dates_deaths_m, color='blue')
    plt.plot(day_index_m, pol_reg_m.predict(poly_reg_m.fit_transform(day_index_m)), color='blue')
    plt.title('Polynomial Reg')
    plt.xlabel('Zeitindex in Tagen')
    plt.ylabel('Gemeldete Tode')
    plt.show()
    return
viz_polymonial()

## Compare regression recoveries M/F

In [ ]:
get_degree(day_index_m,dates_recovered_m) #All Data

In [ ]:
get_degree(day_index_f,dates_recovered_f) #All Data

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
plt.clf()
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
poly_reg = PolynomialFeatures(degree=14) #Same

X_poly_f = poly_reg.fit_transform(day_index_f)
pol_reg_f = LinearRegression()
pol_reg_f.fit(X_poly_f, dates_recovered_f)

X_poly_m = poly_reg.fit_transform(day_index_m)
pol_reg_m = LinearRegression()
pol_reg_m.fit(X_poly_m, dates_recovered_m)

fig, ax = plt.subplots(figsize = (20, 15))

def viz_polymonial():
    plt.scatter(day_index_f, dates_recovered_f, color='red')
    plt.plot(day_index_f, pol_reg_f.predict(poly_reg.fit_transform(day_index_f)), color='red')
    plt.scatter(day_index_m, dates_recovered_m, color='blue')
    plt.plot(day_index_m, pol_reg_m.predict(poly_reg.fit_transform(day_index_m)), color='blue')
    plt.title('Polynomial Reg')
    plt.xlabel('Zeitindex in Tagen')
    plt.ylabel('Gemeldete Genesungen')
    plt.show()
    return
viz_polymonial()

## => Behaviour of regression curves between cases and recoveries between M/F mostly identical. Differences can be visually identiefied between the deaths.

# **2. Non-Graphical analytics of the dataset(s)**
## Non aggregated data

# Kurtosis
### Using Fisher: Norm => 0.0

In [ ]:
print('Kurtosis cases: {}'.format(kurtosis(sorted_df['cases'], fisher=True)))
print('Kurtosis deaths: {}'.format(kurtosis(sorted_df['deaths'], fisher=True)))
print('Kurtosis recoveries: {}'.format(kurtosis(sorted_df['recovered'], fisher=True)))

### Using Pearson: Norm => 3.0

In [ ]:
print('Kurtosis cases: {}'.format(kurtosis(sorted_df['cases'], fisher=False)))
print('Kurtosis deaths: {}'.format(kurtosis(sorted_df['deaths'], fisher=False)))
print('Kurtosis recoveries: {}'.format(kurtosis(sorted_df['recovered'], fisher=False)))

## => Extremely Leptokurtic 

# Skew

In [ ]:
print('Skew Cases: %.3f'%skew(sorted_df['cases']))
print('Skew Recovered: %.3f'%skew(sorted_df['recovered']))
print('Skew Deaths: %.3f'%skew(sorted_df['deaths']))

## => Right Skewed data

# Check for type of destribution. Specify H0 hypothesis as: Data come from [insert type of distribution here] distribution

In [ ]:
print('Test whether cases data is normal disrtibuted: {}'.format(scipy.stats.anderson(sorted_df['cases'], dist='norm')))
print('Test whether cases data is gumbal distributed: {}'.format(scipy.stats.anderson(sorted_df['cases'], dist='gumbel')))
print('Test whether cases data is exponential distributed: {}'.format(scipy.stats.anderson(sorted_df['cases'], dist='expon')))
print('Test whether cases data is logisic distributed: {}'.format(scipy.stats.anderson(sorted_df['cases'], dist='logistic')))
print('##########################################################################')
print('Test whether deaths data is normal disrtibuted: {}'.format(scipy.stats.anderson(sorted_df['deaths'], dist='norm')))
print('Test whether deaths data is gumbal distributed: {}'.format(scipy.stats.anderson(sorted_df['deaths'], dist='gumbel')))
print('Test whether deaths data is exponential distributed: {}'.format(scipy.stats.anderson(sorted_df['deaths'], dist='expon')))
print('Test whether deaths data is logisic distributed: {}'.format(scipy.stats.anderson(sorted_df['deaths'], dist='logistic')))
print('##########################################################################')
print('Test whether revoered data is normal disrtibuted: {}'.format(scipy.stats.anderson(sorted_df['recovered'], dist='norm')))
print('Test whether recovered data is gumbal distributed: {}'.format(scipy.stats.anderson(sorted_df['recovered'], dist='gumbel')))
print('Test whether recovered data is exponential distributed: {}'.format(scipy.stats.anderson(sorted_df['recovered'], dist='expon')))
print('Test whether recovered data is logisic distributed: {}'.format(scipy.stats.anderson(sorted_df['recovered'], dist='logistic')))

## => Continuous reject H0 and choosing alternative hypothesis H1 that data does not follow given distributions

# Kolmogorov-Smirnov test with hypothesis H0: Data come from normal distributed population

In [ ]:
Mean = np.mean(sorted_df['cases'])
STD = np.std(sorted_df['cases'])
k_cases = scipy.stats.kstest(sorted_df['cases'], 'norm', args=(Mean, STD))

Mean = np.mean(sorted_df['deaths'])
STD = np.std(sorted_df['deaths'])
k_deaths = scipy.stats.kstest(sorted_df['deaths'], 'norm', args=(Mean, STD))

Mean = np.mean(sorted_df['recovered'])
STD = np.std(sorted_df['recovered'])
k_rec = scipy.stats.kstest(sorted_df['recovered'], 'norm', args=(Mean, STD))

print('K-Test cases: {}'.format(k_cases))
print('K-Test deaths: {}'.format(k_deaths))
print('K-Test recoveries: {}'.format(k_rec))

## => Continuous reject H0 and choosing alternative hypothesis H1 that data does not come from normal distributed population

# Check normal distribution using D'Agostino Omnibus Test (H0/H1 stays same)

In [ ]:
stat, p = normaltest(sorted_df['cases'])
print('Statistics=%.3f, p=%.3f' % (stat, p)) 

alpha = 0.05
if p > alpha:
	print('Cases data is normal distributed (Can not reject H0)') 
else:
	print('Cases data is not normal distributed (Reject H0)') 
print('##########################################################################')
stat, p = normaltest(sorted_df['deaths']) 
print('Statistics=%.3f, p=%.3f' % (stat, p)) 

alpha = 0.05 
if p > alpha:
	print('Deaths data is normal distributed (Can not reject H0)')
else:
	print('Deaths data is not normal distributed (Reject H0)')
print('##########################################################################')
stat, p = normaltest(sorted_df['recovered'])
print('Statistics=%.3f, p=%.3f' % (stat, p)) 

alpha = 0.05
if p > alpha:
	print('Recoveries data is normal distributed (Can not reject H0)')
else:
	print('Recoveries data is not normal distributed (Reject H0)')

# Finally use lilliefors test for verify that H0 is not suitable

In [ ]:
print('Cases: {}'.format(lilliefors(sorted_df['cases'], pvalmethod='table')))
print('Deaths: {}'.format(lilliefors(sorted_df['deaths'], pvalmethod='table')))
print('Recoveries: {}'.format(lilliefors(sorted_df['recovered'], pvalmethod='table')))

## P-Value always < alpha=0.05. Reject H0

# Analyze correlation
### Positive correlations imply that as x increases, so does y. Negative correlations imply that as x increases, y decreases.

In [ ]:
print('Pearson correlation cases/recoveries: {}'.format(pearsonr(sorted_df['cases'],sorted_df['recovered'])[0]))
print('Pearson correlation cases/deaths: {}'.format(pearsonr(sorted_df['cases'],sorted_df['deaths'])[0]))
print('Pearson correlation deaths/recoveries: {}'.format(pearsonr(sorted_df['deaths'],sorted_df['recovered'])[0]))

In [ ]:
sorted_df.corr()

## => Non aggregated data shows mostly no correlation except cases/recovered

# **2.1 Use aggregated dataset**
## => Aggregate cases/deaths/recoveries from all Germany for each unique day.
## => Hypothesis tests share same results. So doesn't wrote them down.

# Kurtosis
## Fisher...see section 2. 

In [ ]:
print('Kurtosis cases aggreg. data: {}'.format(kurtosis(dates_cases, fisher=True)))
print('Kurtosis deaths aggreg. data: {}'.format(kurtosis(dates_death, fisher=True)))
print('Kurtosis recoveries aggreg. data: {}'.format(kurtosis(dates_recovered, fisher=True)))

## Pearson...see section 2.

In [ ]:
print('Kurtosis cases aggreg. data: {}'.format(kurtosis(dates_cases, fisher=False)))
print('Kurtosis deaths aggreg. data: {}'.format(kurtosis(dates_death, fisher=False)))
print('Kurtosis recoveries aggreg. data: {}'.format(kurtosis(dates_recovered, fisher=False)))

## => Cases/Recoveries mostly STD, Platykurtic 
## => Deaths seems like hyperbolic or laplace, Leptokurtic


# Skew

In [ ]:
print('Skew Cases: %.3f'%skew(dates_cases))
print('Skew Recovered: %.3f'%skew(dates_death))
print('Skew Deaths: %.3f'%skew(dates_recovered))

## => Right Skewed data

# Analyze correlation aggregated data
### Positive correlations imply that as x increases, so does y. Negative correlations imply that as x increases, y decreases.

In [ ]:
print('Pearson correlation cases/recoveries: {}'.format(pearsonr(dates_cases,dates_recovered)[0]))
print('Pearson correlation cases/deaths: {}'.format(pearsonr(dates_cases,dates_death)[0]))
print('Pearson correlation deaths/recoveries: {}'.format(pearsonr(dates_death,dates_recovered)[0]))

## => Stronger correlation of aggregated data towards non aggregated data

# **2.2 Use aggregated sub-datasets M/F**
# Quick representation of sub-datasets

In [ ]:
df_male.describe(include = [np.number])

In [ ]:
df_male.describe(include=['object'])

In [ ]:
df_female.describe(include = [np.number])

In [ ]:
df_female.describe(include=['object'])

# Use T-Test for check Hypothesis H0 that both sub-datasets share same average of data (μX=μY). NON AGGREGATED DATA!
## Measure variance for check whether using Welch-Test or not (use Welch-Test when sub-datasets variances differ)

In [ ]:
print('Difference variances cases: {}'.format(math.sqrt(math.pow(np.var(df_male['cases'])-np.var(df_female['cases']),2))))
print('Difference variances deaths: {}'.format(math.sqrt(math.pow(np.var(df_male['deaths'])-np.var(df_female['deaths']),2))))
print('Difference variances recoveries: {}'.format(math.sqrt(math.pow(np.var(df_male['recovered'])-np.var(df_female['recovered']),2))))

In [ ]:
print('T-Statistic cases M/F: {}'.format(stats.ttest_ind(df_male['cases'], df_female['cases'], equal_var=False))) #Welch-Test
print('T-Statistic deaths M/F: {}'.format(stats.ttest_ind(df_male['deaths'], df_female['deaths'], equal_var=True)))
print('T-Statistic recoveries M/F: {}'.format(stats.ttest_ind(df_male['recovered'], df_female['recovered'], equal_var=False))) #Welch-Test

## => P-Value way too low, reject H0 => μX≠μY

# Use T-Test for check Hypothesis H0 that both sub-datasets share same average of data (μX=μY). AGGREGATED DATA!
## Measure variance for check whether using Welch-Test or not (use Welch-Test when sub-datasets variances differ)

In [ ]:
print('Difference variances cases: {}'.format(math.sqrt(math.pow(np.var(np.array(dates_cases_m))-np.var(np.array(dates_cases_f)),2))))
print('Difference variances deaths: {}'.format(math.sqrt(math.pow(np.var(np.array(dates_deaths_m))-np.var(np.array(dates_deaths_f)),2))))
print('Difference variances recoveries: {}'.format(math.sqrt(math.pow(np.var(np.array(dates_recovered_m))-np.var(np.array(dates_recovered_f)),2))))



In [ ]:
print('T-Statistic cases M/F: {}'.format(stats.ttest_ind(np.array(dates_cases_m), np.array(dates_cases_f), equal_var=False))) #Welch-Test
print('T-Statistic deaths M/F: {}'.format(stats.ttest_ind(np.array(dates_deaths_m), np.array(dates_deaths_f), equal_var=True))) #Welch-Test
print('T-Statistic recoveries M/F: {}'.format(stats.ttest_ind(np.array(dates_recovered_m), np.array(dates_recovered_f), equal_var=False))) #Welch-Test

## => P-Value abote alpha=0.05 => Failed to reject H0. Gender specific sub-datasets share same averages WHEN AGGREGATED!

# Analyze correlation aggregated gender data
### Positive correlations imply that as x increases, so does y. Negative correlations imply that as x increases, y decreases.

In [ ]:
print(np.array(dates_cases_m).shape)
print(np.array(dates_cases_f).shape)

In [ ]:
print(np.array(dates_deaths_m).shape)
print(np.array(dates_deaths_f).shape)

In [ ]:
print(np.array(dates_recovered_m).shape)
print(np.array(dates_recovered_f).shape)

# Trim larger dataset to match length of smaller dataset
## Yet just implemented case: dataset male > dataset female!

In [ ]:
dates_cases_m = dates_cases_m[:np.array(dates_recovered_f).shape[0]]
dates_deaths_m = dates_deaths_m[:np.array(dates_deaths_f).shape[0]]
dates_recovered_m = dates_recovered_m[:np.array(dates_recovered_f).shape[0]]

In [ ]:
print('Korrelation nach Pearson zwischen Neuinfektionen M/W: {}'.format(pearsonr(dates_cases_m,dates_cases_f)[0]))
print('Korrelation nach Pearson zwischen Tode M/W: {}'.format(pearsonr(dates_deaths_m,dates_deaths_f)[0]))
print('Korrelation nach Pearson zwischen Genesungen M/W: {}'.format(pearsonr(dates_recovered_m,dates_recovered_f)[0]))


## => There is a measureable correlation between the gender specific sub-datasets

In [ ]:
arr_m = [np.array(dates_cases_m), np.array(dates_deaths_m), np.array(dates_recovered_m)]
arr_f = [np.array(dates_cases_f), np.array(dates_deaths_f), np.array(dates_recovered_f)]

In [ ]:
print('Order x,y = Cases, Deaths, Recovered')
np.corrcoef(arr_m) # Male

In [ ]:
print('Reihenfolge x,y = Cases, Deaths, Recovered')
np.corrcoef(arr_f) # Female

# Pearsons Chi Square to check independency of data
## Cases

In [ ]:
prob = 0.95
contigency= pd.crosstab(np.array(dates_cases_m), np.array(dates_cases_f))
contigency

In [ ]:
stat, p, dof, expected = chi2_contingency(contigency) 
chi2_contingency(contigency) 


critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
	print('Dependent (Reject H0)')
else:
	print('Independent (Can not reject H0)')
# Interpretiere Hypothese anhand des p-Wertes
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
	print('Dependent (Reject H0)')
else:
	print('Independent (Can not reject H0)')

## Calculate Camers V Cases

In [ ]:
X2 = stat
data = [np.array(dates_cases_m),np.array(dates_cases_f)]
n = np.sum(data)
#Choose smaller dimension: x or y => x
minDim = 452-1

#Berechne Cramer's V
V = np.sqrt((X2/n) / minDim)

print('Cramers V: {}'.format(V))

## Deaths

In [ ]:
prob = 0.95
contigency= pd.crosstab(np.array(dates_deaths_m), np.array(dates_deaths_f))
contigency

In [ ]:
stat, p, dof, expected = chi2_contingency(contigency) 
chi2_contingency(contigency) 


critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
	print('Dependent (Reject H0)')
else:
	print('Independent (Can not reject H0)')
# Interpretiere Hypothese anhand des p-Wertes
alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
	print('Dependent (Reject H0)')
else:
	print('Independent (Can not reject H0)')

## Calculate Cramers V Deaths

In [ ]:
X2 = stat
data = [np.array(dates_deaths_m),np.array(dates_deaths_f)]
n = np.sum(data)
#Choose smaller dimension: x or y => x
minDim = 452-1

#Berechne Cramer's V
V = np.sqrt((X2/n) / minDim)

print('Cramers V: {}'.format(V))

## Recoveries

In [ ]:
prob = 0.95
contigency= pd.crosstab(np.array(dates_recovered_m), np.array(dates_recovered_f))
contigency

In [ ]:
stat, p, dof, expected = chi2_contingency(contigency) 
chi2_contingency(contigency) 


critical = chi2.ppf(prob, dof)
print('probability=%.3f, critical=%.3f, stat=%.3f' % (prob, critical, stat))
if abs(stat) >= critical:
	print('Dependent (Reject H0)')
else:
	print('Independent (Can not reject H0)')

alpha = 1.0 - prob
print('significance=%.3f, p=%.3f' % (alpha, p))
if p <= alpha:
	print('Dependent (Reject H0)')
else:
	print('Independent (Can not reject H0)')

## Calculate Cramers V Recoveries

In [ ]:
X2 = stat
data = [np.array(dates_recovered_m),np.array(dates_recovered_f)]
n = np.sum(data)
#Choose smaller dimension: x or y => x
minDim = 452-1

V = np.sqrt((X2/n) / minDim)

print('Cramers V: {}'.format(V))

# Wilcoxon-Test (alpha = 0.05)

In [ ]:
from scipy.stats import ranksums, wilcoxon
ranksums(np.array(dates_cases_m), np.array(dates_cases_f))

In [ ]:
ranksums(np.array(dates_deaths_m), np.array(dates_deaths_f))

In [ ]:
ranksums(np.array(dates_recovered_m), np.array(dates_recovered_f))

In [ ]:
wilcoxon(np.array(dates_deaths_m), np.array(dates_deaths_f),alternative="greater")

In [ ]:
wilcoxon(np.array(dates_deaths_m), np.array(dates_deaths_f))

In [ ]:
wilcoxon(np.array(dates_deaths_f), np.array(dates_deaths_m),alternative="greater")

In [ ]:
wilcoxon(np.array(dates_recovered_m), np.array(dates_recovered_f))